In [3]:
def get_all_cities(data):
    cleaned_data = data.dropna()
    # make a separate dataframe for each city
    amaravati = cleaned_data[cleaned_data.City == 'Amaravati']
    amritsar = cleaned_data[cleaned_data.City == 'Amritsar']
    chandigarh = cleaned_data[cleaned_data.City == 'Chandigarh']
    delhi = cleaned_data[cleaned_data.City == 'Delhi']
    gurugram = cleaned_data[cleaned_data.City == 'Gurugram']
    hyderabad = cleaned_data[cleaned_data.City == 'Hyderabad']
    kolkata = cleaned_data[cleaned_data.City == 'Kolkata']
    patna = cleaned_data[cleaned_data.City == 'Patna']
    visakhapatnam = cleaned_data[cleaned_data.City == 'Visakhapatnam']
    return amaravati, amritsar, chandigarh, delhi, gurugram, hyderabad, kolkata, patna, visakhapatnam

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, RobustScaler, StandardScaler

def preprocess_data(X, y, scale_method = 'default'):
    if scale_method == 'default':
        y = y/y.max()
        X = X/X.max()
    else:
        if scale_method == 'standard':
            scaler = StandardScaler()
        elif scale_method == 'minmax':
            scaler = MinMaxScaler()
        elif scale_method == 'maxabs':
            scaler = MaxAbsScaler()
        elif scale_method == 'robust':
            scaler = RobustScaler()
        else:
            raise ValueError("Invalid scaling method")
        X = scaler.fit_transform(X)
        y = y.values
        y = y.reshape(-1,1)
        y = scaler.fit_transform(y)
        
    if scale_method == 'default':
        return X, y, None
    else:
        return X, y, scaler
    
def clean_data(data):
    cleaned_data = data.dropna()
    gasses = cleaned_data.select_dtypes(include = np.float64)
    corr = gasses.corr().AQI
    col_to_drop = corr[abs(corr) < 0.45].index
    gasses = gasses.drop(columns = col_to_drop)
    y = gasses.AQI
    X = gasses.drop(columns = 'AQI')
    return X, y

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import os

def print_model_performance(mse, r2, mae, mape, model_name = 'Linear Regression', scale_method = 'default'):
    print("\n For Model: {} with Scale Method: {}".format(model_name, scale_method))
    print("Mean Squared Error:", mse)
    print("R-squared:", r2)
    print("Mean Absolute Error:", mae)
    print("Mean Absolute Percentage Error:", mape)
    
def log_results(model_name, scale_method, split, mse, r2, mae, mape, city=""):
    # initialize an empty df to concat into later
    base_df = pd.DataFrame(columns = ['Model', 'Scale Method', 'Data Split', 'City', 'MSE', 'R2', 'MAE', 'MAPE'])
    # in the df we store the model name, the scaling method, datasplit, and the mae, mse, r2, and mape
    new_df = pd.DataFrame([[model_name, scale_method, split, city, mse, r2, mae, mape]], columns = ['Model', 'Scale Method', 'Data Split', 'City', 'MSE', 'R2', 'MAE', 'MAPE'])
    # if the file exists, we read it in and concat the new results
    if os.path.exists('patchresults.csv'):
        base_df = pd.read_csv('patchresults.csv')
        base_df = pd.concat([base_df, new_df])
        base_df.to_csv('patchresults.csv', index = False)
    # if the file doesn't exist, we create a new one
    else:
        new_df.to_csv('patchresults.csv', index = False)

def plot_predictions(y_test, y_pred, scale_method = 'default', model_name = 'Linear Regression', split=0.3, city = ""):
    plt.figure(figsize=(10, 5))
    y_test = np.array(y_test)
    y_pred = np.array(y_pred)
    y_test = y_test[:int(split * len(y_test))]
    y_pred = y_pred[:int(split * len(y_pred))]
    plt.plot(y_test, label='True')
    plt.plot(y_pred, label='Predicted')
    plt.title("True vs Predicted AQI of {} using {} with Scale Method: {}".format(city, model_name, scale_method))
    plt.xlabel("Observation")
    plt.ylabel("AQI")
    plt.legend()
    plt.show()

d:\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [6]:
def linear_regression_model(data, scale_method = 'default', plot_split=0.3, data_split=0.2, city=""):
    X, y = clean_data(data)
    # y_max = y.max()
    X, y, scaler = preprocess_data(X, y, scale_method)
    X_train = X[:int(data_split * len(X))]
    X_test = X[int(data_split * len(X)):]
    y_train = y[:int(data_split * len(y))]
    y_test = y[int(data_split * len(y)):]
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print_model_performance(mse, r2, mae, mape, model_name='Linear Regression', scale_method=scale_method)
    plot_predictions(y_test, y_pred, scale_method=scale_method, model_name='Linear Regression', split=plot_split, city=city)
    log_results('Linear Regression', scale_method, data_split, mse, r2, mae, mape, city=city)

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

def nn_model(data, scale_method = 'default', plot_split = 0.3, data_split=0.2, city=""):
    X, y = clean_data(data)
    # y_max = y.max()
    X, y, scaler = preprocess_data(X, y, scale_method)
    X_train = X[:int(data_split * len(X))]
    X_test = X[int(data_split * len(X)):]
    y_train = y[:int(data_split * len(y))]
    y_test = y[int(data_split * len(y)):]
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[es], verbose=0)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)   
    mae = mean_absolute_error(y_test, y_pred)
    try:
        mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    except:
        # convert to one dimensional array
        y_test = np.array(y_test)
        y_pred = np.array(y_pred)
        mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print_model_performance(mse, r2, mae, mape, model_name='Neural Network', scale_method=scale_method)
    plot_predictions(y_test, y_pred, scale_method=scale_method, model_name='Neural Network', split=plot_split, city=city)
    log_results('Neural Network', scale_method, data_split, mse, r2, mae, mape, city=city)
    

In [8]:
from keras.layers import LSTM
from keras.layers import Dropout

def lstm_model(data, scale_method = 'maxabs', plot_split = 0.3, data_split=0.2, embedding_size = 256, dropout = False, city=""):
    if scale_method == 'default':
        print("Default scaling method not supported for LSTM... switching to maxabs")
        scale_method = 'maxabs'
    X, y = clean_data(data)
    # y_max = y.max()
    X, y, scaler = preprocess_data(X, y, scale_method)
    X_train = X[:int(data_split * len(X))]
    X_test = X[int(data_split * len(X)):]
    y_train = y[:int(data_split * len(y))]
    y_test = y[int(data_split * len(y)):]
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    model = Sequential()
    model.add(LSTM(embedding_size, input_shape=(X_train.shape[1], X_train.shape[2]),))
    if dropout:
        model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[es], verbose=0)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print_model_performance(mse, r2, mae, mape, model_name='LSTM', scale_method=scale_method)
    plot_predictions(y_test, y_pred, scale_method=scale_method, model_name='LSTM', split=plot_split, city=city)
    log_results('LSTM', scale_method, data_split, mse, r2, mae, mape, city=city)

In [9]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
def arima_model(data, scale_method = 'minmax', plot_split = 0.3, data_split=0.2, order=(5,1,0), city=""):
    if scale_method == 'default':
        raise ValueError("SARIMAX model does not support 'default' scaling method")
    X, y = clean_data(data)
    # y_max = y.max()
    X, y, scaler = preprocess_data(X, y, scale_method)
    X_train = X[:int(data_split * len(X))]
    X_test = X[int(data_split * len(X)):]
    y_train = y[:int(data_split * len(y))]
    y_test = y[int(data_split * len(y)):]
    model = SARIMAX(y_train, order=order)
    model_fit = model.fit(disp=0)
    y_pred = model_fit.predict(start=len(y_train), end=len(y_train)+len(y_test)-1, dynamic=False)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print_model_performance(mse, r2, mae, mape, model_name='SARIMAX', scale_method=scale_method)
    plot_predictions(y_test, y_pred, scale_method=scale_method, model_name='SARIMAX', split=plot_split, city=city)
    log_results('SARIMAX', scale_method, data_split, mse, r2, mae, mape, city=city)

In [10]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

def holt_winters_model(data, scale_method = 'default', plot_split = 0.3, data_split=0.2, city=""):
    # if scale_method == 'default':
    #     raise ValueError("Holt-Winters model does not support 'default' scaling method")
    X, y = clean_data(data)
    # y_max = y.max()
    X, y, scaler = preprocess_data(X, y, scale_method)
    X_train = X[:int(data_split * len(X))]
    X_test = X[int(data_split * len(X)):]
    y_train = y[:int(data_split * len(y))]
    y_test = y[int(data_split * len(y)):]
    model = ExponentialSmoothing(y_train, trend='add', seasonal='add', seasonal_periods=366)
    model_fit = model.fit()
    y_pred = model_fit.predict(start=len(y_train), end=len(y_train)+len(y_test)-1)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print_model_performance(mse, r2, mae, mape, model_name='Holt-Winters', scale_method=scale_method)
    plot_predictions(y_test, y_pred, model_name='Holt-Winters', scale_method=scale_method, split=plot_split, city=city)
    log_results('Holt-Winters', scale_method, data_split, mse, r2, mae, mape, city=city)

In [11]:
# implemment RNN model 
from keras.layers import SimpleRNN

def rnn_model(data, scale_method = 'default', plot_split = 0.3, data_split=0.2, city=""):
    if scale_method == 'default':
        print("RNN model does not support 'default' scaling method. Using 'minmax' scaling method") 
        scale_method = 'minmax'
    X, y = clean_data(data)
    # y_max = y.max()
    X, y, scaler = preprocess_data(X, y, scale_method)
    X_train = X[:int(data_split * len(X))]
    X_test = X[int(data_split * len(X)):]
    y_train = y[:int(data_split * len(y))]
    y_test = y[int(data_split * len(y)):]
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    model = Sequential()
    model.add(SimpleRNN(312, input_shape=(X_train.shape[1], X_train.shape[2]),))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[es], verbose=0)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print_model_performance(mse, r2, mae, mape, model_name='RNN', scale_method=scale_method)
    plot_predictions(y_test, y_pred, scale_method=scale_method, model_name='RNN', split=plot_split, city=city)
    log_results('RNN', scale_method, data_split, mse, r2, mae, mape, city=city)

In [12]:
# implement LTC regressor
import torch
import torch.nn as nn
from ncps.wirings import AutoNCP
from ncps.torch import LTC
import pytorch_lightning as pl
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

class SequenceLearner(pl.LightningModule):
    def __init__(self, model, lr=0.005):
        super().__init__()
        self.model = model
        self.lr = lr

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat, _ = self.model.forward(x)
        y_hat = y_hat.view_as(y)
        loss = nn.MSELoss()(y_hat, y)
        self.log("train_loss", loss, prog_bar=True)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat, _ = self.model.forward(x)
        y_hat = y_hat.view_as(y)
        loss = nn.MSELoss()(y_hat, y)

        self.log("val_loss", loss, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.lr)

def ltc_model(data, scale_method = 'maxabs', plot_split = 0.3, data_split=0.2, embedding_size = 256, dropout = False, city=""):
    # if scale_method == 'default':
    #     print("Default scaling method not supported for LTC... switching to maxabs")
    #     scale_method = 'maxabs'
    X, y = clean_data(data)
    X, y, scaler = preprocess_data(X, y, scale_method)
    X_train = X[:int(data_split * len(X))]
    X_test = X[int(data_split * len(X)):]
    y_train = y[:int(data_split * len(y))]
    y_test = y[int(data_split * len(y)):]
    X_train = torch.Tensor(X_train)
    X_test = torch.Tensor(X_test)
    y_train = torch.Tensor(y_train)
    y_test = torch.Tensor(y_test)
    dataloader = DataLoader(
        TensorDataset(X_train, y_train), batch_size=1, shuffle=True, num_workers=4
    )
    out_features = 1
    in_features = X_train.shape[1]
    wiring = AutoNCP(embedding_size, out_features)
    ltc_model = LTC(in_features, wiring, batch_first=True)
    learn = SequenceLearner(ltc_model, lr=0.01)
    trainer = pl.Trainer(
        max_epochs=100,
        gradient_clip_val=1,
    )
    trainer.fit(learn, dataloader)
    y_pred, _ = ltc_model.forward(X_test)
    y_pred = y_pred.detach().numpy()
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    # mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    mape=0
    print_model_performance(mse, r2, mae, mape, model_name='LTC', scale_method=scale_method)
    plot_predictions(y_test, y_pred, scale_method=scale_method, model_name='LTC', split=plot_split, city=city)
    log_results('LTC', scale_method, data_split, mse, r2, mae, mape, city=city)

In [28]:
from typing import Callable, Optional
import torch
from torch import nn
from torch import Tensor
import torch.nn.functional as F
import numpy as np

import torch
from torch import nn
import math

class Transpose(nn.Module):
    def __init__(self, *dims, contiguous=False): 
        super().__init__()
        self.dims, self.contiguous = dims, contiguous
    def forward(self, x):
        if self.contiguous: return x.transpose(*self.dims).contiguous()
        else: return x.transpose(*self.dims)

    
def get_activation_fn(activation):
    if callable(activation): return activation()
    elif activation.lower() == "relu": return nn.ReLU()
    elif activation.lower() == "gelu": return nn.GELU()
    raise ValueError(f'{activation} is not available. You can use "relu", "gelu", or a callable') 

class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

def PositionalEncoding(q_len, d_model, normalize=True):
    pe = torch.zeros(q_len, d_model)
    position = torch.arange(0, q_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    if normalize:
        pe = pe - pe.mean()
        pe = pe / (pe.std() * 10)
    return pe

SinCosPosEncoding = PositionalEncoding

def Coord2dPosEncoding(q_len, d_model, exponential=False, normalize=True, eps=1e-3, verbose=False):
    x = .5 if exponential else 1
    i = 0
    for i in range(100):
        cpe = 2 * (torch.linspace(0, 1, q_len).reshape(-1, 1) ** x) * (torch.linspace(0, 1, d_model).reshape(1, -1) ** x) - 1
        pv(f'{i:4.0f}  {x:5.3f}  {cpe.mean():+6.3f}', verbose)
        if abs(cpe.mean()) <= eps: break
        elif cpe.mean() > eps: x += .001
        else: x -= .001
        i += 1
    if normalize:
        cpe = cpe - cpe.mean()
        cpe = cpe / (cpe.std() * 10)
    return cpe

def Coord1dPosEncoding(q_len, exponential=False, normalize=True):
    cpe = (2 * (torch.linspace(0, 1, q_len).reshape(-1, 1)**(.5 if exponential else 1)) - 1)
    if normalize:
        cpe = cpe - cpe.mean()
        cpe = cpe / (cpe.std() * 10)
    return cpe

def positional_encoding(pe, learn_pe, q_len, d_model):
    # Positional encoding
    if pe == None:
        W_pos = torch.empty((q_len, d_model)) # pe = None and learn_pe = False can be used to measure impact of pe
        nn.init.uniform_(W_pos, -0.02, 0.02)
        learn_pe = False
    elif pe == 'zero':
        W_pos = torch.empty((q_len, 1))
        nn.init.uniform_(W_pos, -0.02, 0.02)
    elif pe == 'zeros':
        W_pos = torch.empty((q_len, d_model))
        nn.init.uniform_(W_pos, -0.02, 0.02)
    elif pe == 'normal' or pe == 'gauss':
        W_pos = torch.zeros((q_len, 1))
        torch.nn.init.normal_(W_pos, mean=0.0, std=0.1)
    elif pe == 'uniform':
        W_pos = torch.zeros((q_len, 1))
        nn.init.uniform_(W_pos, a=0.0, b=0.1)
    elif pe == 'lin1d': W_pos = Coord1dPosEncoding(q_len, exponential=False, normalize=True)
    elif pe == 'exp1d': W_pos = Coord1dPosEncoding(q_len, exponential=True, normalize=True)
    elif pe == 'lin2d': W_pos = Coord2dPosEncoding(q_len, d_model, exponential=False, normalize=True)
    elif pe == 'exp2d': W_pos = Coord2dPosEncoding(q_len, d_model, exponential=True, normalize=True)
    elif pe == 'sincos': W_pos = PositionalEncoding(q_len, d_model, normalize=True)
    else: raise ValueError(f"{pe} is not a valid pe (positional encoder. Available types: 'gauss'=='normal', \
        'zeros', 'zero', uniform', 'lin1d', 'exp1d', 'lin2d', 'exp2d', 'sincos', None.)")
    return nn.Parameter(W_pos, requires_grad=learn_pe)

from typing import Callable, Optional
import torch
from torch import nn
from torch import Tensor
import torch.nn.functional as F
import numpy as np
import torch
import torch.nn as nn

class RevIN(nn.Module):
    def __init__(self, num_features: int, eps=1e-5, affine=True, subtract_last=False):
        """
        :param num_features: the number of features or channels
        :param eps: a value added for numerical stability
        :param affine: if True, RevIN has learnable affine parameters
        """
        super(RevIN, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.affine = affine
        self.subtract_last = subtract_last
        if self.affine:
            self._init_params()

    def forward(self, x, mode:str):
        if mode == 'norm':
            self._get_statistics(x)
            x = self._normalize(x)
        elif mode == 'denorm':
            x = self._denormalize(x)
        else: raise NotImplementedError
        return x

    def _init_params(self):
        # initialize RevIN params: (C,)
        self.affine_weight = nn.Parameter(torch.ones(self.num_features))
        self.affine_bias = nn.Parameter(torch.zeros(self.num_features))

    def _get_statistics(self, x):
        dim2reduce = tuple(range(1, x.ndim-1))
        if self.subtract_last:
            self.last = x[:,-1,:].unsqueeze(1)
        else:
            self.mean = torch.mean(x, dim=dim2reduce, keepdim=True).detach()
        self.stdev = torch.sqrt(torch.var(x, dim=dim2reduce, keepdim=True, unbiased=False) + self.eps).detach()

    def _normalize(self, x):
        if self.subtract_last:
            x = x - self.last
        else:
            x = x - self.mean
        x = x / self.stdev
        if self.affine:
            x = x * self.affine_weight
            x = x + self.affine_bias
        return x

    def _denormalize(self, x):
        if self.affine:
            x = x - self.affine_bias
            x = x / (self.affine_weight + self.eps*self.eps)
        x = x * self.stdev
        if self.subtract_last:
            x = x + self.last
        else:
            x = x + self.mean
        return x

# Cell
class PatchTST_backbone(nn.Module):
    def __init__(self, c_in:int, context_window:int, target_window:int, patch_len:int, stride:int, max_seq_len:Optional[int]=1024, 
                 n_layers:int=3, d_model=128, n_heads=16, d_k:Optional[int]=None, d_v:Optional[int]=None,
                 d_ff:int=256, norm:str='BatchNorm', attn_dropout:float=0., dropout:float=0., act:str="gelu", key_padding_mask:bool='auto',
                 padding_var:Optional[int]=None, attn_mask:Optional[Tensor]=None, res_attention:bool=True, pre_norm:bool=False, store_attn:bool=False,
                 pe:str='zeros', learn_pe:bool=True, fc_dropout:float=0., head_dropout = 0, padding_patch = None,
                 pretrain_head:bool=False, head_type = 'flatten', individual = False, revin = True, affine = True, subtract_last = False,
                 verbose:bool=False, **kwargs):
        
        super().__init__()
        
        # RevIn
        self.revin = revin
        if self.revin: self.revin_layer = RevIN(c_in, affine=affine, subtract_last=subtract_last)
        
        # Patching
        self.patch_len = patch_len
        self.stride = stride
        self.padding_patch = padding_patch
        patch_num = int((context_window - patch_len)/stride + 1)
        if padding_patch == 'end': # can be modified to general case
            self.padding_patch_layer = nn.ReplicationPad1d((0, stride)) 
            patch_num += 1
        
        # Backbone 
        self.backbone = TSTiEncoder(c_in, patch_num=patch_num, patch_len=patch_len, max_seq_len=max_seq_len,
                                n_layers=n_layers, d_model=d_model, n_heads=n_heads, d_k=d_k, d_v=d_v, d_ff=d_ff,
                                attn_dropout=attn_dropout, dropout=dropout, act=act, key_padding_mask=key_padding_mask, padding_var=padding_var,
                                attn_mask=attn_mask, res_attention=res_attention, pre_norm=pre_norm, store_attn=store_attn,
                                pe=pe, learn_pe=learn_pe, verbose=verbose, **kwargs)

        # Head
        self.head_nf = d_model * patch_num
        self.n_vars = c_in
        self.pretrain_head = pretrain_head
        self.head_type = head_type
        self.individual = individual

        if self.pretrain_head: 
            self.head = self.create_pretrain_head(self.head_nf, c_in, fc_dropout) # custom head passed as a partial func with all its kwargs
        elif head_type == 'flatten': 
            self.head = Flatten_Head(self.individual, self.n_vars, self.head_nf, target_window, head_dropout=head_dropout)
        
    
    def forward(self, z):                                                                   # z: [bs x nvars x seq_len]
        # norm
        if self.revin: 
            z = z.permute(0,2,1)
            z = self.revin_layer(z, 'norm')
            z = z.permute(0,2,1)
            
        # do patching
        if self.padding_patch == 'end':
            z = self.padding_patch_layer(z)
        z = z.unfold(dimension=-1, size=self.patch_len, step=self.stride)                   # z: [bs x nvars x patch_num x patch_len]
        z = z.permute(0,1,3,2)                                                              # z: [bs x nvars x patch_len x patch_num]
        
        # model
        z = self.backbone(z)                                                                # z: [bs x nvars x d_model x patch_num]
        z = self.head(z)                                                                    # z: [bs x nvars x target_window] 
        
        # denorm
        if self.revin: 
            z = z.permute(0,2,1)
            z = self.revin_layer(z, 'denorm')
            z = z.permute(0,2,1)
        return z
    
    def create_pretrain_head(self, head_nf, vars, dropout):
        return nn.Sequential(nn.Dropout(dropout),
                    nn.Conv1d(head_nf, vars, 1)
                    )

class Flatten_Head(nn.Module):
    def __init__(self, individual, n_vars, nf, target_window, head_dropout=0):
        super().__init__()
        
        self.individual = individual
        self.n_vars = n_vars
        
        if self.individual:
            self.linears = nn.ModuleList()
            self.dropouts = nn.ModuleList()
            self.flattens = nn.ModuleList()
            for i in range(self.n_vars):
                self.flattens.append(nn.Flatten(start_dim=-2))
                self.linears.append(nn.Linear(nf, target_window))
                self.dropouts.append(nn.Dropout(head_dropout))
        else:
            self.flatten = nn.Flatten(start_dim=-2)
            self.linear = nn.Linear(nf, target_window)
            self.dropout = nn.Dropout(head_dropout)
            
    def forward(self, x):                                 # x: [bs x nvars x d_model x patch_num]
        if self.individual:
            x_out = []
            for i in range(self.n_vars):
                z = self.flattens[i](x[:,i,:,:])          # z: [bs x d_model * patch_num]
                z = self.linears[i](z)                    # z: [bs x target_window]
                z = self.dropouts[i](z)
                x_out.append(z)
            x = torch.stack(x_out, dim=1)                 # x: [bs x nvars x target_window]
        else:
            x = self.flatten(x)
            x = self.linear(x)
            x = self.dropout(x)
        return x
        
    
class TSTiEncoder(nn.Module):  #i means channel-independent
    def __init__(self, c_in, patch_num, patch_len, max_seq_len=1024,
                 n_layers=3, d_model=128, n_heads=16, d_k=None, d_v=None,
                 d_ff=256, norm='BatchNorm', attn_dropout=0., dropout=0., act="gelu", store_attn=False,
                 key_padding_mask='auto', padding_var=None, attn_mask=None, res_attention=True, pre_norm=False,
                 pe='zeros', learn_pe=True, verbose=False, **kwargs):
        
        
        super().__init__()
        
        self.patch_num = patch_num
        self.patch_len = patch_len
        
        # Input encoding
        q_len = patch_num
        self.W_P = nn.Linear(patch_len, d_model)        # Eq 1: projection of feature vectors onto a d-dim vector space
        self.seq_len = q_len

        # Positional encoding
        self.W_pos = positional_encoding(pe, learn_pe, q_len, d_model)

        # Residual dropout
        self.dropout = nn.Dropout(dropout)

        # Encoder
        self.encoder = TSTEncoder(q_len, d_model, n_heads, d_k=d_k, d_v=d_v, d_ff=d_ff, norm=norm, attn_dropout=attn_dropout, dropout=dropout,
                                   pre_norm=pre_norm, activation=act, res_attention=res_attention, n_layers=n_layers, store_attn=store_attn)

        
    def forward(self, x) -> Tensor:                                              # x: [bs x nvars x patch_len x patch_num]
        
        n_vars = x.shape[1]
        # Input encoding
        x = x.permute(0,1,3,2)                                                   # x: [bs x nvars x patch_num x patch_len]
        x = self.W_P(x)                                                          # x: [bs x nvars x patch_num x d_model]

        u = torch.reshape(x, (x.shape[0]*x.shape[1],x.shape[2],x.shape[3]))      # u: [bs * nvars x patch_num x d_model]
        u = self.dropout(u + self.W_pos)                                         # u: [bs * nvars x patch_num x d_model]

        # Encoder
        z = self.encoder(u)                                                      # z: [bs * nvars x patch_num x d_model]
        z = torch.reshape(z, (-1,n_vars,z.shape[-2],z.shape[-1]))                # z: [bs x nvars x patch_num x d_model]
        z = z.permute(0,1,3,2)                                                   # z: [bs x nvars x d_model x patch_num]
        
        return z    
            
            
    
# Cell
class TSTEncoder(nn.Module):
    def __init__(self, q_len, d_model, n_heads, d_k=None, d_v=None, d_ff=None, 
                        norm='BatchNorm', attn_dropout=0., dropout=0., activation='gelu',
                        res_attention=False, n_layers=1, pre_norm=False, store_attn=False):
        super().__init__()

        self.layers = nn.ModuleList([TSTEncoderLayer(q_len, d_model, n_heads=n_heads, d_k=d_k, d_v=d_v, d_ff=d_ff, norm=norm,
                                                      attn_dropout=attn_dropout, dropout=dropout,
                                                      activation=activation, res_attention=res_attention,
                                                      pre_norm=pre_norm, store_attn=store_attn) for i in range(n_layers)])
        self.res_attention = res_attention

    def forward(self, src:Tensor, key_padding_mask:Optional[Tensor]=None, attn_mask:Optional[Tensor]=None):
        output = src
        scores = None
        if self.res_attention:
            for mod in self.layers: output, scores = mod(output, prev=scores, key_padding_mask=key_padding_mask, attn_mask=attn_mask)
            return output
        else:
            for mod in self.layers: output = mod(output, key_padding_mask=key_padding_mask, attn_mask=attn_mask)
            return output



class TSTEncoderLayer(nn.Module):
    def __init__(self, q_len, d_model, n_heads, d_k=None, d_v=None, d_ff=256, store_attn=False,
                 norm='BatchNorm', attn_dropout=0, dropout=0., bias=True, activation="gelu", res_attention=False, pre_norm=False):
        super().__init__()
        assert not d_model%n_heads, f"d_model ({d_model}) must be divisible by n_heads ({n_heads})"
        d_k = d_model // n_heads if d_k is None else d_k
        d_v = d_model // n_heads if d_v is None else d_v

        # Multi-Head attention
        self.res_attention = res_attention
        self.self_attn = _MultiheadAttention(d_model, n_heads, d_k, d_v, attn_dropout=attn_dropout, proj_dropout=dropout, res_attention=res_attention)

        # Add & Norm
        self.dropout_attn = nn.Dropout(dropout)
        if "batch" in norm.lower():
            self.norm_attn = nn.Sequential(Transpose(1,2), nn.BatchNorm1d(d_model), Transpose(1,2))
        else:
            self.norm_attn = nn.LayerNorm(d_model)

        # Position-wise Feed-Forward
        self.ff = nn.Sequential(nn.Linear(d_model, d_ff, bias=bias),
                                get_activation_fn(activation),
                                nn.Dropout(dropout),
                                nn.Linear(d_ff, d_model, bias=bias))

        # Add & Norm
        self.dropout_ffn = nn.Dropout(dropout)
        if "batch" in norm.lower():
            self.norm_ffn = nn.Sequential(Transpose(1,2), nn.BatchNorm1d(d_model), Transpose(1,2))
        else:
            self.norm_ffn = nn.LayerNorm(d_model)

        self.pre_norm = pre_norm
        self.store_attn = store_attn


    def forward(self, src:Tensor, prev:Optional[Tensor]=None, key_padding_mask:Optional[Tensor]=None, attn_mask:Optional[Tensor]=None) -> Tensor:

        # Multi-Head attention sublayer
        if self.pre_norm:
            src = self.norm_attn(src)
        ## Multi-Head attention
        if self.res_attention:
            src2, attn, scores = self.self_attn(src, src, src, prev, key_padding_mask=key_padding_mask, attn_mask=attn_mask)
        else:
            src2, attn = self.self_attn(src, src, src, key_padding_mask=key_padding_mask, attn_mask=attn_mask)
        if self.store_attn:
            self.attn = attn
        ## Add & Norm
        src = src + self.dropout_attn(src2) # Add: residual connection with residual dropout
        if not self.pre_norm:
            src = self.norm_attn(src)

        # Feed-forward sublayer
        if self.pre_norm:
            src = self.norm_ffn(src)
        ## Position-wise Feed-Forward
        src2 = self.ff(src)
        ## Add & Norm
        src = src + self.dropout_ffn(src2) # Add: residual connection with residual dropout
        if not self.pre_norm:
            src = self.norm_ffn(src)

        if self.res_attention:
            return src, scores
        else:
            return src




class _MultiheadAttention(nn.Module):
    def __init__(self, d_model, n_heads, d_k=None, d_v=None, res_attention=False, attn_dropout=0., proj_dropout=0., qkv_bias=True, lsa=False):
        """Multi Head Attention Layer
        Input shape:
            Q:       [batch_size (bs) x max_q_len x d_model]
            K, V:    [batch_size (bs) x q_len x d_model]
            mask:    [q_len x q_len]
        """
        super().__init__()
        d_k = d_model // n_heads if d_k is None else d_k
        d_v = d_model // n_heads if d_v is None else d_v

        self.n_heads, self.d_k, self.d_v = n_heads, d_k, d_v

        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=qkv_bias)
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=qkv_bias)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=qkv_bias)

        # Scaled Dot-Product Attention (multiple heads)
        self.res_attention = res_attention
        self.sdp_attn = _ScaledDotProductAttention(d_model, n_heads, attn_dropout=attn_dropout, res_attention=self.res_attention, lsa=lsa)

        # Poject output
        self.to_out = nn.Sequential(nn.Linear(n_heads * d_v, d_model), nn.Dropout(proj_dropout))


    def forward(self, Q:Tensor, K:Optional[Tensor]=None, V:Optional[Tensor]=None, prev:Optional[Tensor]=None,
                key_padding_mask:Optional[Tensor]=None, attn_mask:Optional[Tensor]=None):

        bs = Q.size(0)
        if K is None: K = Q
        if V is None: V = Q

        # Linear (+ split in multiple heads)
        q_s = self.W_Q(Q).view(bs, -1, self.n_heads, self.d_k).transpose(1,2)       # q_s    : [bs x n_heads x max_q_len x d_k]
        k_s = self.W_K(K).view(bs, -1, self.n_heads, self.d_k).permute(0,2,3,1)     # k_s    : [bs x n_heads x d_k x q_len] - transpose(1,2) + transpose(2,3)
        v_s = self.W_V(V).view(bs, -1, self.n_heads, self.d_v).transpose(1,2)       # v_s    : [bs x n_heads x q_len x d_v]

        # Apply Scaled Dot-Product Attention (multiple heads)
        if self.res_attention:
            output, attn_weights, attn_scores = self.sdp_attn(q_s, k_s, v_s, prev=prev, key_padding_mask=key_padding_mask, attn_mask=attn_mask)
        else:
            output, attn_weights = self.sdp_attn(q_s, k_s, v_s, key_padding_mask=key_padding_mask, attn_mask=attn_mask)
        # output: [bs x n_heads x q_len x d_v], attn: [bs x n_heads x q_len x q_len], scores: [bs x n_heads x max_q_len x q_len]

        # back to the original inputs dimensions
        output = output.transpose(1, 2).contiguous().view(bs, -1, self.n_heads * self.d_v) # output: [bs x q_len x n_heads * d_v]
        output = self.to_out(output)

        if self.res_attention: return output, attn_weights, attn_scores
        else: return output, attn_weights


class _ScaledDotProductAttention(nn.Module):
    r"""Scaled Dot-Product Attention module (Attention is all you need by Vaswani et al., 2017) with optional residual attention from previous layer
    (Realformer: Transformer likes residual attention by He et al, 2020) and locality self sttention (Vision Transformer for Small-Size Datasets
    by Lee et al, 2021)"""

    def __init__(self, d_model, n_heads, attn_dropout=0., res_attention=False, lsa=False):
        super().__init__()
        self.attn_dropout = nn.Dropout(attn_dropout)
        self.res_attention = res_attention
        head_dim = d_model // n_heads
        self.scale = nn.Parameter(torch.tensor(head_dim ** -0.5), requires_grad=lsa)
        self.lsa = lsa

    def forward(self, q:Tensor, k:Tensor, v:Tensor, prev:Optional[Tensor]=None, key_padding_mask:Optional[Tensor]=None, attn_mask:Optional[Tensor]=None):
        '''
        Input shape:
            q               : [bs x n_heads x max_q_len x d_k]
            k               : [bs x n_heads x d_k x seq_len]
            v               : [bs x n_heads x seq_len x d_v]
            prev            : [bs x n_heads x q_len x seq_len]
            key_padding_mask: [bs x seq_len]
            attn_mask       : [1 x seq_len x seq_len]
        Output shape:
            output:  [bs x n_heads x q_len x d_v]
            attn   : [bs x n_heads x q_len x seq_len]
            scores : [bs x n_heads x q_len x seq_len]
        '''

        # Scaled MatMul (q, k) - similarity scores for all pairs of positions in an input sequence
        attn_scores = torch.matmul(q, k) * self.scale      # attn_scores : [bs x n_heads x max_q_len x q_len]

        # Add pre-softmax attention scores from the previous layer (optional)
        if prev is not None: attn_scores = attn_scores + prev

        # Attention mask (optional)
        if attn_mask is not None:                                     # attn_mask with shape [q_len x seq_len] - only used when q_len == seq_len
            if attn_mask.dtype == torch.bool:
                attn_scores.masked_fill_(attn_mask, -np.inf)
            else:
                attn_scores += attn_mask

        # Key padding mask (optional)
        if key_padding_mask is not None:                              # mask with shape [bs x q_len] (only when max_w_len == q_len)
            attn_scores.masked_fill_(key_padding_mask.unsqueeze(1).unsqueeze(2), -np.inf)

        # normalize the attention weights
        attn_weights = F.softmax(attn_scores, dim=-1)                 # attn_weights   : [bs x n_heads x max_q_len x q_len]
        attn_weights = self.attn_dropout(attn_weights)

        # compute the new values given the attention weights
        output = torch.matmul(attn_weights, v)                        # output: [bs x n_heads x max_q_len x d_v]

        if self.res_attention: return output, attn_weights, attn_scores
        else: return output, attn_weights

class Model(nn.Module):
    def __init__(self, configs, max_seq_len:Optional[int]=1024, d_k:Optional[int]=None, d_v:Optional[int]=None, norm:str='BatchNorm', attn_dropout:float=0., 
                 act:str="gelu", key_padding_mask:bool='auto',padding_var:Optional[int]=None, attn_mask:Optional[Tensor]=None, res_attention:bool=True, 
                 pre_norm:bool=False, store_attn:bool=False, pe:str='zeros', learn_pe:bool=True, pretrain_head:bool=False, head_type = 'flatten', verbose:bool=False, **kwargs):
        
        super().__init__()
        
        # load parameters
        c_in = configs.enc_in
        context_window = configs.seq_len
        target_window = configs.pred_len
        
        n_layers = configs.e_layers
        n_heads = configs.n_heads
        d_model = configs.d_model
        d_ff = configs.d_ff
        dropout = configs.dropout
        fc_dropout = configs.fc_dropout
        head_dropout = configs.head_dropout
        
        individual = configs.individual
    
        patch_len = configs.patch_len
        stride = configs.stride
        padding_patch = configs.padding_patch
        
        revin = configs.revin
        affine = configs.affine
        subtract_last = configs.subtract_last
        
        decomposition = configs.decomposition
        kernel_size = configs.kernel_size
        
        
        # model
        self.decomposition = decomposition
        if self.decomposition:
            self.decomp_module = series_decomp(kernel_size)
            self.model_trend = PatchTST_backbone(c_in=c_in, context_window = context_window, target_window=target_window, patch_len=patch_len, stride=stride, 
                                  max_seq_len=max_seq_len, n_layers=n_layers, d_model=d_model,
                                  n_heads=n_heads, d_k=d_k, d_v=d_v, d_ff=d_ff, norm=norm, attn_dropout=attn_dropout,
                                  dropout=dropout, act=act, key_padding_mask=key_padding_mask, padding_var=padding_var, 
                                  attn_mask=attn_mask, res_attention=res_attention, pre_norm=pre_norm, store_attn=store_attn,
                                  pe=pe, learn_pe=learn_pe, fc_dropout=fc_dropout, head_dropout=head_dropout, padding_patch = padding_patch,
                                  pretrain_head=pretrain_head, head_type=head_type, individual=individual, revin=revin, affine=affine,
                                  subtract_last=subtract_last, verbose=verbose, **kwargs)
            self.model_res = PatchTST_backbone(c_in=c_in, context_window = context_window, target_window=target_window, patch_len=patch_len, stride=stride, 
                                  max_seq_len=max_seq_len, n_layers=n_layers, d_model=d_model,
                                  n_heads=n_heads, d_k=d_k, d_v=d_v, d_ff=d_ff, norm=norm, attn_dropout=attn_dropout,
                                  dropout=dropout, act=act, key_padding_mask=key_padding_mask, padding_var=padding_var, 
                                  attn_mask=attn_mask, res_attention=res_attention, pre_norm=pre_norm, store_attn=store_attn,
                                  pe=pe, learn_pe=learn_pe, fc_dropout=fc_dropout, head_dropout=head_dropout, padding_patch = padding_patch,
                                  pretrain_head=pretrain_head, head_type=head_type, individual=individual, revin=revin, affine=affine,
                                  subtract_last=subtract_last, verbose=verbose, **kwargs)
        else:
            self.model = PatchTST_backbone(c_in=c_in, context_window = context_window, target_window=target_window, patch_len=patch_len, stride=stride, 
                                  max_seq_len=max_seq_len, n_layers=n_layers, d_model=d_model,
                                  n_heads=n_heads, d_k=d_k, d_v=d_v, d_ff=d_ff, norm=norm, attn_dropout=attn_dropout,
                                  dropout=dropout, act=act, key_padding_mask=key_padding_mask, padding_var=padding_var, 
                                  attn_mask=attn_mask, res_attention=res_attention, pre_norm=pre_norm, store_attn=store_attn,
                                  pe=pe, learn_pe=learn_pe, fc_dropout=fc_dropout, head_dropout=head_dropout, padding_patch = padding_patch,
                                  pretrain_head=pretrain_head, head_type=head_type, individual=individual, revin=revin, affine=affine,
                                  subtract_last=subtract_last, verbose=verbose, **kwargs)
    
    
    def forward(self, x):           # x: [Batch, Input length, Channel]
        if self.decomposition:
            res_init, trend_init = self.decomp_module(x)
            res_init, trend_init = res_init.permute(0,2,1), trend_init.permute(0,2,1)  # x: [Batch, Channel, Input length]
            res = self.model_res(res_init)
            trend = self.model_trend(trend_init)
            x = res + trend
            x = x.permute(0,2,1)    # x: [Batch, Input length, Channel]
        else:
            x = x.permute(0,2,1)    # x: [Batch, Channel, Input length]
            x = self.model(x)
            x = x.permute(0,2,1)    # x: [Batch, Input length, Channel]
        return x
    
def patch_tst_model(data, configs, scale_method = 'default', plot_split = 0.3, data_split=0.2, city=""):
    if scale_method == 'default':
        print("PatchTST model does not support 'default' scaling method.")
        scale_method = "minmax"
    X, y = clean_data(data)
    X, y, scaler = preprocess_data(X, y, scale_method)
    X_train = X[:int(data_split * len(X))]
    X_test = X[int(data_split * len(X)):]
    y_train = y[:int(data_split * len(y))]
    y_test = y[int(data_split * len(y)):]
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    # X_train = torch.Tensor(X_train, dtype=torch.float32)
    # X_test = torch.Tensor(X_test, dtype=torch.float32)
    # y_train = torch.Tensor(y_train, dtype=torch.float32)
    # y_test = torch.Tensor(y_test, dtype=torch.float32)
    model = Model(configs)
    optimizer = torch.optim.Adam(model.parameters())
    criterion = torch.nn.MSELoss()
    for epoch in range(200):
        model.train()
        optimizer.zero_grad()
        outputs = model(torch.from_numpy(X_train).float())
        loss = criterion(outputs, torch.from_numpy(y_train).float())
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch+1}, Loss: {loss.item()}')
    y_pred = model(torch.from_numpy(X_test).float()).detach().numpy()
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print_model_performance(mse, r2, mae, mape, model_name='PatchTST', scale_method=scale_method)
    plot_predictions(y_test, y_pred, scale_method=scale_method, model_name='PatchTST', split=plot_split, city=city)
    log_results('PatchTST', scale_method, data_split, mse, r2, mae, mape, city=city)
    
class Configs:
    def __init__(self):
        self.enc_in = 10  # number of features
        self.seq_len = 100  # length of the sequence
        self.pred_len = 1  # prediction horizon
        self.e_layers = 3  # number of encoder layers
        self.n_heads = 8  # number of attention heads
        self.d_model = 512  # dimension of the model
        self.d_ff = 2048  # dimension of the feedforward network
        self.dropout = 0.1  # dropout rate
        self.fc_dropout = 0.1  # dropout rate for the fully connected layer
        self.head_dropout = 0.1  # dropout rate for the head
        self.individual = False  # whether to use individual normalization
        self.patch_len = 1  # length of the patches
        self.stride = 1  # stride for the patches
        self.padding_patch = 0  # padding for the patches
        self.revin = False  # whether to reverse the input sequence
        self.affine = True  # whether to use affine transformation in the normalization layers
        self.subtract_last = False  # whether to subtract the last value in each patch
        self.decomposition = False  # whether to use series decomposition
        self.kernel_size = 5  # kernel size for the series decomposition

In [29]:
import pandas as pd
def run_all_methods_on_model(data, model_name, data_split=0.2, plot_split=0.3, city=""):
    scalers = ['default', 'standard', 'minmax', 'maxabs', 'robust']
    configs = Configs()
    for scaler in scalers:
        # if model_name == 'sarimax':
        #     if scaler == 'default':
        #         print("default scaler not supported for SARIMAX model...switching to standard scaler")
        #         scaler = 'standard'
        #     arima_model(data, scaler, data_split=data_split, plot_split=plot_split, city=city)
        # elif model_name == 'holt_winters':
        #     holt_winters_model(data, scaler, data_split=data_split, plot_split=plot_split, city=city)
        if model_name == 'linear':
            linear_regression_model(data, scaler, data_split=data_split, plot_split=plot_split, city=city)
        elif model_name == 'nn':
            nn_model(data, scaler, data_split=data_split, plot_split=plot_split, city=city)
        elif model_name == 'lstm':
            lstm_model(data, scaler, data_split=data_split, plot_split=plot_split, city=city)
        elif model_name == 'rnn':
            rnn_model(data, scaler, data_split=data_split, plot_split=plot_split, city=city)
        elif model_name == 'ltc':
            ltc_model(data, scaler, data_split=data_split, plot_split=plot_split, city=city)
        elif model_name == 'patchtst':
            patch_tst_model(data, configs, scaler, data_split=data_split, plot_split=plot_split, city=city)
        else:
            raise ValueError("Invalid model type")
        pass

In [30]:
city_day = pd.read_csv('city_day.csv')
amaravati, amritsar, chandigarh, delhi, gurugram, hyderabad, kolkata, patna, visakhapatnam = get_all_cities(city_day)

In [31]:
# models = ['linear', 'nn', 'lstm', 'rnn',]
# for model in models:
#     run_all_methods_on_model(delhi, model, data_split=0.2, plot_split=0.3, city="Delhi")
    # run_all_methods_on_model(gurugram, model, data_split=0.2, plot_split=0.3, city="Gurugram")
    # run_all_methods_on_model(hyderabad, model, data_split=0.2, plot_split=0.3, city="Hyderabad")
    # run_all_methods_on_model(kolkata, model, data_split=0.2, plot_split=0.3, city="Kolkata")
    # run_all_methods_on_model(patna, model, data_split=0.2, plot_split=0.3, city="Patna")
    # run_all_methods_on_model(visakhapatnam, model, data_split=0.2, plot_split=0.3, city="Visakhapatnam")
    # run_all_methods_on_model(amaravati, model, data_split=0.2, plot_split=0.3, city="Amaravati")
    # run_all_methods_on_model(amritsar, model, data_split=0.2, plot_split=0.3, city="Amritsar")
    # run_all_methods_on_model(chandigarh, model, data_split=0.2, plot_split=0.3, city="Chandigarh")

In [32]:
run_all_methods_on_model(delhi, 'patchtst', data_split=0.2, plot_split=0.3, city="Delhi")

PatchTST model does not support 'default' scaling method.
